In [78]:
import boto3
from botocore.exceptions import ClientError
import time
import os
import paramiko
from subnets.subnets import get_subnets

#referencia: https://github.com/Otofuji/orm

In [79]:
ec2 = boto3.client('ec2')
ec2.describe_instances()

{'Reservations': [{'Groups': [],
   'Instances': [{'AmiLaunchIndex': 0,
     'ImageId': 'ami-0ed9277fb7eb570c9',
     'InstanceId': 'i-0de42ab5253ffb751',
     'InstanceType': 't2.micro',
     'KeyName': 'key-us-east-1',
     'LaunchTime': datetime.datetime(2021, 12, 3, 22, 47, 40, tzinfo=tzutc()),
     'Monitoring': {'State': 'disabled'},
     'Placement': {'AvailabilityZone': 'us-east-1b',
      'GroupName': '',
      'Tenancy': 'default'},
     'PrivateDnsName': 'ip-172-31-89-117.ec2.internal',
     'PrivateIpAddress': '172.31.89.117',
     'ProductCodes': [],
     'PublicDnsName': 'ec2-174-129-146-0.compute-1.amazonaws.com',
     'PublicIpAddress': '174.129.146.0',
     'State': {'Code': 16, 'Name': 'running'},
     'StateTransitionReason': '',
     'SubnetId': 'subnet-9d308eb3',
     'VpcId': 'vpc-5f9a5725',
     'Architecture': 'x86_64',
     'BlockDeviceMappings': [{'DeviceName': '/dev/xvda',
       'Ebs': {'AttachTime': datetime.datetime(2021, 12, 3, 22, 47, 41, tzinfo=tzutc())

In [80]:
#Create Key
def cria_Key(region):
    ec2_region = boto3.client('ec2')
    Keyname = "key-"+region
    
    keys = ec2_region.describe_key_pairs()
    for key in list(keys.values())[0]:
        print(key["KeyName"])
        if (key["KeyName"] == Keyname):
            print("Deletando par de chaves existente...")
            deleted = ec2_region.delete_key_pair(KeyName=Keyname)
            os.remove("./" + Keyname + ".pem")

    resp = ec2.create_key_pair(KeyName = Keyname)
    Filename = Keyname+'.pem'
            
    file = open(Filename,'w')
    file.write(resp['KeyMaterial'])
    file.close
    print(f'Key criada em {Filename}')
    return resp

In [81]:
#apaga instances
def apaga_instances(region):
    processando = False
    
    SG_name="SG-"+region
    
    ec2_region = boto3.client('ec2', region_name = region)

    print("Apagando Instances")
    instances = ec2_region.describe_instances()
    n_instances = len(instances['Reservations'])
    for i in range(n_instances-1):
        #if True:
        try:
            print(i)
            instance_SG = instances['Reservations'][i]['Instances'][0]['NetworkInterfaces'][0]['Groups'][0]['GroupName']
            print(instance_SG)
            if instance_SG == SG_name:
                instance_id = instances['Reservations'][i]['Instances'][0]['InstanceId']
                ec2_region.terminate_instances(InstanceIds = [instance_id])
                processando = True
                print(f"Apagou uma instância em {region}")
        except:
            pass
        
    while processando:
        time.sleep(5)
        processando = False
        instances = ec2_region.describe_instances()
        for i in range(n_instances):
            try:
                instance_SG = instances['Reservations'][i]['Instances'][0]['NetworkInterfaces'][0]['Groups'][0]['GroupName']
                if instance_SG == SG_name:
                    processando = True
                    print("Aguardando")
            except:
                pass
        time.sleep(5)
    print(f"Finalizou Instances em {region}")
    return

def apaga_SG(region):
    processando = False
    
    SG_name="SG-"+region
    
    ec2_region = boto3.client('ec2', region_name = region)

        #Apaga SGs
    print("Apagando Security Groups")
    SGs = ec2_region.describe_security_groups()['SecurityGroups']
    n_SGs = len(SGs)

    for i in range (n_SGs):
        try:
        #if True:
            #print(ec2_region.describe_security_groups()["SecurityGroups"][i]["GroupName"])
            if (ec2_region.describe_security_groups()["SecurityGroups"][i]["GroupName"] == SG_name):
                print(ec2_region.describe_security_groups()["SecurityGroups"][i]["GroupName"])
                ec2_region.delete_security_group(GroupName = SG_name)
                print(f"Apagou Security Group de {region}")
                break
        except:
            print("Não foi possível apagar o Security Group")
    print(f"Deletou Security Groups em {region}")
    return #ec2_region
    

def apaga_existentes(region):
    apaga_instances(region)
    apaga_SG(region)
    return

In [82]:
#Create Security Group
#Create Security Group
def cria_SG(region):
    ec2_region = boto3.client('ec2', region_name = region)
    
    SG_name="SG-"+region
    print(f"Criando Security Group em {region}")
    
    #ec2_region.delete_security_group(GroupName = SG_name)
    
    vpcs = ec2_region.describe_vpcs()
    vpc_id = vpcs.get('Vpcs', [{}])[0].get('VpcId', '')
    vpc = vpcs.get('Vpcs', [{}])[0]
    try:
    
        resp = ec2.create_security_group(
            GroupName = SG_name,
            Description = 'grupo '+region,
            VpcId = vpc_id
        )
        SG_id = resp['GroupId']
        print(f"SG_ID ====="{SG_id}")
        print(f"{SG_name} {SG_id} criado em {vpc_id}")
        data_in = ec2_region.authorize_security_group_ingress(
            GroupId = SG_id,
            IpPermissions = [
                {'IpProtocol': 'tcp',
                'FromPort': 80,
                'ToPort': 80,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                {'IpProtocol': 'tcp',
                'FromPort': 5432,
                'ToPort': 5432,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                {'IpProtocol': 'tcp',
                'FromPort': 8800,
                'ToPort': 8800,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                {'IpProtocol': 'tcp',
                'FromPort': 22,
                'ToPort': 22,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
            ])
        data_out = ec2_region.authorize_security_group_egress(
            GroupId = SG_id,
            IpPermissions = [
                {'IpProtocol': 'tcp',
                'FromPort': 80,
                'ToPort': 80,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                {'IpProtocol': 'tcp',
                'FromPort': 5432,
                'ToPort': 5432,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                {'IpProtocol': 'tcp',
                'FromPort': 8800,
                'ToPort': 8800,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                {'IpProtocol': 'tcp',
                'FromPort': 22,
                'ToPort': 22,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
            ])
        print("Security Group criado")
        return resp 
    except ClientError as err:
        print(err)
        
    return SG_id #ec2_region

SyntaxError: invalid syntax (Temp/ipykernel_11400/3203465490.py, line 22)

In [ ]:
#Create EC2
def cria_instance(region,user_data,img):
    SG_name="SG-"+region
    Keyname="key-"+region
    
    ec2_region = boto3.client('ec2', region_name = region)
    
    #configura instancia

    print("userdata:")
    print(user_data)

    print(SG_name)
    ec2_resource = boto3.resource('ec2', region_name = region)
    instances = ec2_resource.create_instances(
        #Ubuntu Server 18.04 LTS (HVM), SSD Volume Type - ami-0279c3b3186e54acd (64 bits x86) 
        ImageId = img,
        UserData = user_data,
        MinCount = 1,
        MaxCount = 1,
        InstanceType = 't2.micro',
        KeyName = Keyname,
        BlockDeviceMappings = [
            {
                'DeviceName' : "/dev/xvda",
                'Ebs' :{
                    'DeleteOnTermination': True,
                    'VolumeSize': 20
                }
            }
        ],
        SecurityGroups = [SG_name]
    )
    print(f"criou nova instance em {region}")
    
    return instances[0]

In [86]:
def configure_instance(instance,region,user_data,postgresIP):
    filename = "./key-"+region+'.pem'
    Keyname = "key-"+region
    
    print("Aguardando a instância rodar")
    instance.wait_until_running()
    instance.load()
    print(instance.public_dns_name)
    DNSname: str = "ubuntu@" + instance.public_dns_name
    print(DNSname)
    postgresIP = str(instance.public_ip_address)
    user_data = user_data.replace("postgresIP", postgresIP)
    time.sleep(30)
    
    k = paramiko.RSAKey.from_private_key_file("./"+filename)
    c = paramiko.SSHClient()
    c.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    print ("connecting")

    c.connect( hostname = instance.public_dns_name, port = 22,  username = "ubuntu", password=None, pkey = k, key_filename = None, timeout=None,  allow_agent=True, look_for_keys=True, compress=False, sock=None, gss_auth=False, gss_kex=False, gss_deleg_creds=True, gss_host=None, banner_timeout=None, auth_timeout=None, gss_trust_dns=True, passphrase=None, disabled_algorithms=None )
    print ("connected")
    print("Config")
    commands = [""]
    for command in commands:
        stdin , stdout, stderr = c.exec_command(command)
        print(stdout.read())
        print(stderr.read())
        
    c.close()
    print("Comandos de configuração enviados para a instância")
    return instance

In [85]:
#config userdata
user_data_ohio = str("""#cloud-config
    runcmd:
    - cd /home/ubuntu;
    - sudo apt update;
    - sudo apt upgrade -y;
    - sudo apt autoremove -y;
    - sudo apt install postgresql postgresql-contrib -y;
    - sudo su - postgres;
    - sudo -u postgres psql -c "CREATE USER cloud WITH PASSWORD 'cloud';";
    - sudo -u postgres psql -c "CREATE DATABASE tasks;";
    - sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE tasks TO cloud;";
    - sudo echo "listen_addresses = '*'" >> /etc/postgresql/10/main/postgresql.conf;
    - sudo echo "host all all 0.0.0.0/0 trust" >> /etc/postgresql/10/main/pg_hba.conf;
    - sudo ufw allow 5432/tcp -y
    - sudo systemctl restart postgresql
    - sudo reboot;
    """)

user_data_other = str("""#cloud-config
    runcmd:
    - cd /home/ubuntu;
    - sudo apt update;
    - sudo apt upgrade -y;
    - sudo apt autoremove -y;
    - sudo apt install python3-pip -y;
    - pip3 install flask requests django stresspy;
    - git clone https://github.com/raulikeda/tasks;
    - cd tasks;
    - sed -i "s/node1/postgresIP/g" ./portfolio/settings.py;
    - ./install.sh;
    - sudo ufw allow 8080/tcp -y;
    - sudo reboot;
    """)
user_data = [user_data_ohio,user_data_other]

In [ ]:
regions = ["us-east-2","us-east-1"]
imgs = ['ami-020db2c14939a8efb','ami-0279c3b3186e54acd']
instance = ['','']
SG_id = ['','']

In [ ]:
i=0
cria_Key(regions[i])
apaga_existentes(regions[i])
SG_id[i] = cria_SG(regions[i])
instance[i] = cria_instance(regions[i],user_data[i],imgs[i])
instance[i] = configure_instance(instance[i],regions[i],user_data[i],str(instance[0].public_ip_address))

murilo
murilo-oficial
MURILO-NEW-KEY
key-us-east-1
key-us-east-2
Deletando par de chaves existente...
Key criada em key-us-east-2.pem
Apagando Instances
0
SG-us-east-2
Apagou uma instância em us-east-2
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguardando
Aguard

In [ ]:
def cria_AMI(instance,region):
    print("Criando AMI")
    ec2_region = boto3.client('ec2', region_name = region)
    try:
        ami_name = 'ami-marco'
        ami_image = ec2_region.create_image(
            Name = ami_name,
            InstanceId = instance.id,
            NoReboot = False,
            TagSpecifications=[{
                "ResourceType": "image",
                "Tags": [{
                    "Key": "Name",
                    "Value": "DMI"
                }]
            }]
        )

        ec2_region.get_waiter('image_available').wait(ImageIds=[ami_image['ImageId']])
        print("    AMI criada")
    except:
        ami_name: str = 'ami-marco-redundance-' + instance.id
        ami_image = ec2_us_east_1.create_image(
            Name = ami_name,
            InstanceId = instance.id,
            NoReboot = False,
            TagSpecifications=[{
                "ResourceType": "image",
                "Tags": [{
                    "Key": "Name",
                    "Value": "DMI"
                }]
            }]
        )

        ec2_region.get_waiter('image_available').wait(ImageIds=[ami_image['ImageId']])
        print("    AMI criada (redundance mode)")

    return ami_image['ImageId']



In [ ]:
def cria_load_balance(region,SG_id):
    ec2_region = boto3.client('ec2', region_name = region)
    subnets = get_subnets(ec2_region)
    elbv2 = boto3.client('elbv2', region_name=region)
    try:
        lbs = elbv2.describe_load_balancers()
        lbs_arn: str = str(lbs['LoadBalancers'][0]['LoadBalancerArn'])
        print(f"lb_arn = {lbs_arn}")
        free_lb = elbv2.delete_load_balancer(LoadBalancerArn=lb_arn)
        print(free_lb)
        waiter = elbv2.get_waiter('load_balancers_deleted')
        print("Load Balancer apagado")
        time.sleep(60)
    except Exception as e:
        print(e)
        
    #Apaga security groups existentes
    vpcs = elastic_load_balancer.describe_vpcs()
    vpc_id = vpcs.get('Vpcs', [{}])[0].get('VpcId', '')

    print(vpc_id)

    for i in range (SG_amount):
        try:    
            print(elbv2.describe_security_groups()["SecurityGroups"][i]["GroupName"])
            if (elbv2.describe_security_groups()["SecurityGroups"][i]["GroupName"] == "SG-LB"):
                elbv2.delete_security_group(GroupName = "SG-LB")
                print("Apagou Security Group de load balancer")
                break
        except:
            print("Não foi possível apagar o Security Group de LB")
            
    #Cria novos
    print("Criando novo Security Group para Load Balancer")

    try:
        
        resp = elbv2.create_security_group(GroupName='SG-LB',
                                            Description='SG-load-balancer',
                                            VpcId=vpc_id)
        lbsg_id = resp['GroupId']
        print(lbsg_id)
        print(f'{lbsg_id} created in {vpc_id}')

        data_in = elbv2.authorize_security_group_ingress(
            GroupId=lbsg_id,
            IpPermissions=[
                {'IpProtocol': 'tcp',
                'FromPort': 80,
                'ToPort': 80,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                {'IpProtocol': 'tcp',
                'FromPort': 8080,
                'ToPort': 8080,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                {'IpProtocol': 'tcp',
                'FromPort': 5432,
                'ToPort': 5432,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                {'IpProtocol': 'tcp',
                'FromPort': 22,
                'ToPort': 22,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
            ])
        data_out = elastic_load_balancer.authorize_security_group_egress(
            GroupId=load_balancer_security_group_id,
            IpPermissions=[
                {'IpProtocol': 'tcp',
                'FromPort': 80,
                'ToPort': 80,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                {'IpProtocol': 'tcp',
                'FromPort': 8080,
                'ToPort': 8080,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                {'IpProtocol': 'tcp',
                'FromPort': 5432,
                'ToPort': 5432,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                {'IpProtocol': 'tcp',
                'FromPort': 22,
                'ToPort': 22,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
            ])
        print("SG em LB criado")
    except ClientError as e:
        print(e)
        
    resp = elbv2.create_load_balancer(
        Name='marco-lb',
        Subnets=subnets,
        SecurityGroups=[
            SG_id,
        ],
        Scheme='internet-facing',
        Tags=[
            {
                'Key': 'Name',
                'Value': 'load-balancer'
            },
        ],
        
        Type='application',
        IpAddressType='ipv4',
    )
    
    resp = elbv2.create_target_group(
        Name='target',
        Protocol='HTTP',
        Port=8080,
        VpcId = vpc_id,
        TargetType = 'instance',
        HealthCheckPort='8080',
        HealthCheckPath = '/admin/',
        HealthCheckProtocol = 'HTTP')
    target_arns = resp["TargetGroups"][0]["TargetGroupArn"]
    return target_arns

In [ ]:
def cria_autoscaling(region,instance,SG_id,ami_id,target_arns) {
    ec2_region = boto3.client('ec2', region_name = region)
    elbv2 = boto3.client('elbv2', region_name=region)
    lb_arn: str = str(lbs['LoadBalancers'][0]['LoadBalancerArn'])
    
    Keyname = "key-"+region
    
    autoscaling = boto3.client('autoscaling')
    
    resp = autoscaling.create_launch_configuration(
        LaunchConfigurationName='launch_autoscaling',
        ImageId = ami_id,
        KeyName=Keyname,
        SecurityGroups=[
            SG_id,
        ],
        InstanceId=instance.id,
        InstanceMonitoring={
            'Enabled': True
        },
        EbsOptimized=True,
        AssociatePublicIpAddress=True,
        
    )
    
    resp = autoscaling.create_auto_scaling_group(
        AutoScalingGroupName='auto-scaling-group',
        LaunchConfigurationName='launch_autoscaling',
        InstanceId=instance.id,
        MinSize=1,
        MaxSize=10,
        DesiredCapacity=100,
        DefaultCooldown=100,
        AvailabilityZones=[
            'us-east-1a',
            'us-east-1b',
            'us-east-1c',
            'us-east-1d',
            'us-east-1e',
            'us-east-1f'
        ],
        LoadBalancerNames=[
            'marco-lb',
        ],
        ServiceLinkedRoleARN=lb_arn,
    )
    
    auto_scalling.attach_load_balancer_target_groups(
      AutoScalingGroupName='auto_scaling_group',
      TargetGroupARNs=[
        target_group_arn
      ]
    )
    
}

In [ ]:
def cria_cloudwatch(regiao):
    cloudwatch = boto3.client('cloudwatch')
    scale_down = cloudwatch.put_metric_alarm(
        AlarmName='scaledown',
        AlarmDescription='High CPU Utilization',
        ActionsEnabled=True,
        MetricName='CPUUtilization',
        Namespace='ELB',
        Statistic='Average',
        Dimensions=[
            {
                'Name': 'InstanceId',
                'Value': 'i-0bb3fcb4396da630b'
            },
        ],
        Period=60,
        Unit='Seconds',
        EvaluationPeriods=1,
        Threshold=65.0,
        ComparisonOperator='GreaterThanOrEqualToThreshold',
    )


    scale_up = client_cw.put_metric_alarm(
        AlarmName='scaleup',
        AlarmDescription='Low CPU Utilization',
        ActionsEnabled=True,
        MetricName='CPUUtilization',
        Namespace='ELB',
        Statistic='Average',
        Dimensions=[
            {
                'Name': 'InstanceId',
                'Value': 'i-0bb3fcb4396da630b'
            }
        ],
        Period=60,
        Unit='Seconds',
        EvaluationPeriods=2,
        Threshold=20.0,
        ComparisonOperator='LessThanThreshold'   
    )	
	
    response = autoscaling.attach_load_balancers(
        AutoScalingGroupName='project_asg',
        LoadBalancerNames=[
            'projectelb'#,
        ]#,
    )

In [ ]:
i=1
cria_Key(regions[i])
apaga_existentes(regions[i])
cria_SG(regions[i])
instance[i] = cria_instance(regions[i],user_data[i],imgs[i])
instance[i] = configure_instance(instance[i],regions[i],user_data[i])
ami_id = cria_AMI(instance[i],regions[i])
target_arns = cria_load_balance(regions[i],SG_id[i])
cria_autoscaling(regions[i],instance[i],SG_id[i],ami_id,target_arns)

userdata:
#cloud-config
    runcmd:
    - cd /home/ubuntu;
    - sudo apt update;
    - sudo apt upgrade -y;
    - sudo apt autoremove -y;
    - sudo apt install python3-pip -y;
    - pip3 install flask requests django stresspy;
    - git clone https://github.com/raulikeda/tasks;
    - cd tasks;
    - sed -i "s/node1/postgresIP/g" ./portfolio/settings.py;
    - ./install.sh;
    - sudo ufw allow 8080/tcp -y;
    - sudo reboot;
    
criou nova instance em us-east-1


{'GroupId': 'sg-033f0409c29fdc920',
 'ResponseMetadata': {'RequestId': '3ec3026e-a450-4940-89c7-5f0f4689b4c1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3ec3026e-a450-4940-89c7-5f0f4689b4c1',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '283',
   'date': 'Fri, 03 Dec 2021 22:24:34 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

{'GroupId': 'sg-033f0409c29fdc920',
 'ResponseMetadata': {'RequestId': '3ec3026e-a450-4940-89c7-5f0f4689b4c1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3ec3026e-a450-4940-89c7-5f0f4689b4c1',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '283',
   'date': 'Fri, 03 Dec 2021 22:24:34 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

'sg-033f0409c29fdc920'

{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-05af75639f861c8cf',
   'GroupId': 'sg-033f0409c29fdc920',
   'GroupOwnerId': '605467815219',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 80,
   'ToPort': 80,
   'CidrIpv4': '0.0.0.0/0'},
  {'SecurityGroupRuleId': 'sgr-0c0047d2d2616b691',
   'GroupId': 'sg-033f0409c29fdc920',
   'GroupOwnerId': '605467815219',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 22,
   'ToPort': 22,
   'CidrIpv4': '0.0.0.0/0'}],
 'ResponseMetadata': {'RequestId': '051e3673-8afe-463b-85b2-eb005ed847fb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '051e3673-8afe-463b-85b2-eb005ed847fb',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '1124',
   'date': 'Fri, 03 Dec 2021 22:24:58 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

[ec2.Instance(id='i-0f7db7fdd1f346b12')]

{'Reservations': [{'Groups': [],
   'Instances': [{'AmiLaunchIndex': 0,
     'ImageId': 'ami-0ed9277fb7eb570c9',
     'InstanceId': 'i-0a109d41b3a30e2a9',
     'InstanceType': 't2.micro',
     'KeyName': 'exemplo',
     'LaunchTime': datetime.datetime(2021, 12, 3, 15, 10, 28, tzinfo=tzutc()),
     'Monitoring': {'State': 'disabled'},
     'Placement': {'AvailabilityZone': 'us-east-1b',
      'GroupName': '',
      'Tenancy': 'default'},
     'PrivateDnsName': '',
     'ProductCodes': [],
     'PublicDnsName': '',
     'State': {'Code': 48, 'Name': 'terminated'},
     'StateTransitionReason': 'User initiated (2021-12-03 21:58:04 GMT)',
     'Architecture': 'x86_64',
     'BlockDeviceMappings': [],
     'ClientToken': '2c0fcc4f-4362-4722-ab64-f315b312d63b',
     'EbsOptimized': False,
     'EnaSupport': True,
     'Hypervisor': 'xen',
     'NetworkInterfaces': [],
     'RootDeviceName': '/dev/xvda',
     'RootDeviceType': 'ebs',
     'SecurityGroups': [],
     'StateReason': {'Code': 'Cl

ClientError: An error occurred (IncorrectInstanceState) when calling the StopInstances operation: This instance 'i-0a109d41b3a30e2a9' is not in a state from which it can be stopped.

{'StartingInstances': [{'CurrentState': {'Code': 16, 'Name': 'running'},
   'InstanceId': 'i-0a109d41b3a30e2a9',
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': '11c7a595-7090-43b0-b8af-874c8d135d05',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '11c7a595-7090-43b0-b8af-874c8d135d05',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '580',
   'date': 'Fri, 03 Dec 2021 15:12:06 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

#cloud-config
    runcmd:
    - cd /home/ubuntu;
    - sudo apt update;
    - sudo apt upgrade -y;
    - sudo apt autoremove -y;
    - sudo apt install postgresql postgresql-contrib -y;
    - sudo su - postgres;
    - sudo -u postgres psql -c "CREATE USER cloud WITH PASSWORD 'cloud';";
    - sudo -u postgres psql -c "CREATE DATABASE tasks;";
    - sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE tasks TO cloud;";
    - sudo echo "listen_addresses = '*'" >> /etc/postgresql/10/main/postgresql.conf;
    - sudo echo "host all all 0.0.0.0/0 trust" >> /etc/postgresql/10/main/pg_hba.conf;
    - sudo ufw allow 5432/tcp -y
    - sudo systemctl restart postgresql
    - sudo reboot;
    


{'SecurityGroups': [{'Description': 'default VPC security group',
   'GroupName': 'default',
   'IpPermissions': [{'IpProtocol': '-1',
     'IpRanges': [],
     'Ipv6Ranges': [],
     'PrefixListIds': [],
     'UserIdGroupPairs': [{'GroupId': 'sg-ff8a799c',
       'UserId': '605467815219'}]}],
   'OwnerId': '605467815219',
   'GroupId': 'sg-ff8a799c',
   'IpPermissionsEgress': [{'IpProtocol': '-1',
     'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
     'Ipv6Ranges': [],
     'PrefixListIds': [],
     'UserIdGroupPairs': []}],
   'VpcId': 'vpc-525fb139'}],
 'ResponseMetadata': {'RequestId': '987b7f15-d03e-40b8-a76c-dfbfcdc7b35d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '987b7f15-d03e-40b8-a76c-dfbfcdc7b35d',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '1484',
   'date': 'Fri, 03 Dec 2021 23:54:21 GMT',
   'server': 'AmazonEC2'},
  'Retry

{'GroupId': 'sg-0741c4a19262126c7',
 'ResponseMetadata': {'RequestId': 'e9230fec-896a-4b25-aafc-d8812e1a7996',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e9230fec-896a-4b25-aafc-d8812e1a7996',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '283',
   'date': 'Fri, 03 Dec 2021 23:54:42 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}